# Information

Creating a binary text classification model to check if a string of text to the text-to-SQL model or another model

# Install libraries

In [1]:
!pip install transformers datasets scikit-learn torch

^C


# Import libraries

In [2]:
from datasets import load_dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
import shutil
from google.colab import files
import os

# Create dataset

Creating a training and testing dataset from the files.

In [3]:
# Step 1: Define paths for the datasets
train_path = "/content/large_synthetic_sql_dataset.csv"
test_path = "/content/test_synthetic_sql_dataset.csv"

# Load the datasets independently
train_data1 = load_dataset("csv", data_files=train_path)["train"]
test_data1 = load_dataset("csv", data_files=test_path)["train"]  # Adjusted to "train" for test data

# Concatenate the datasets (train + train and test + test)
combined_train_data = concatenate_datasets([train_data1])
combined_test_data = concatenate_datasets([test_data1])

# Combine both train and test datasets into one dataset
datasets_dict = {
    "train": combined_train_data,
    "test": combined_test_data
}

# Verify the combined dataset
print(f"Combined Dataset: {datasets_dict}")
print(f"Number of examples in combined dataset: {len(datasets_dict)}")

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Combined Dataset: {'train': Dataset({
    features: ['text', 'label'],
    num_rows: 30000
}), 'test': Dataset({
    features: ['text', 'label'],
    num_rows: 6000
})}
Number of examples in combined dataset: 2


# Tokenizer

 Setup tokenzier and tokenzier the datasets


In [4]:
# Step 2: Load tokenizer and model
model_name = "distilbert-base-uncased"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Step 3: Tokenize the data
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)

tokenized_datasets = {
    "train": datasets_dict["train"].map(preprocess_function, batched=True),
    "test": datasets_dict["test"].map(preprocess_function, batched=True)
}

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

# Training

Traning the model

In [6]:
# Step 4: Define evaluation metrics
def compute_metrics(pred):
    logits, labels = pred
    predictions = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="binary")
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [7]:
# Step 5: Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    save_strategy="epoch",
    metric_for_best_model="f1",
    greater_is_better=True,
)

# Step 6: Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Step 7: Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-7-adf708b34f21>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.000000,0.000008,1.000000,1.000000,1.000000,1.000000
2,0.000000,0.000002,1.000000,1.000000,1.000000,1.000000
3,0.000000,0.000001,1.000000,1.000000,1.000000,1.000000


TrainOutput(global_step=5625, training_loss=0.002019590549191667, metrics={'train_runtime': 152.6984, 'train_samples_per_second': 589.397, 'train_steps_per_second': 36.837, 'total_flos': 325993988880000.0, 'train_loss': 0.002019590549191667, 'epoch': 3.0})

In [8]:
# Save the model and tokenizer
model.save_pretrained("./sql_classifier_large")
tokenizer.save_pretrained("./sql_classifier_large")

('./sql_classifier_large/tokenizer_config.json',
 './sql_classifier_large/special_tokens_map.json',
 './sql_classifier_large/vocab.txt',
 './sql_classifier_large/added_tokens.json',
 './sql_classifier_large/tokenizer.json')

# Export

Export the model

In [ ]:
# Convert bytes to a more readable format (KB, MB, GB)
def convert_size(size_in_bytes):
    for unit in ['bytes', 'KB', 'MB', 'GB', 'TB']:
        if size_in_bytes < 1024.0:
            return f"{size_in_bytes:.2f} {unit}"
        size_in_bytes /= 1024.0


# Step 1: Define the paths
output_dir = "/content/trained_model"  # The directory where the model is saved
zip_file_path = "/content/trained_model.zip"  # The location for the zip file

# Save the model
model.save_pretrained(output_dir)

# Save the tokenizer
tokenizer.save_pretrained(output_dir)

print(f"Model and tokenizer saved to {output_dir}")

# Step 2: Zip the trained model directory
shutil.make_archive(zip_file_path.replace('.zip', ''), 'zip', output_dir)

# Step 3: Check if the zip file exists and get the file size
if os.path.exists(zip_file_path):
    file_size = os.path.getsize(zip_file_path)
    print(f"File size: {file_size} bytes ({convert_size(file_size)})")
else:
    print("Failed to create the zip file.")

# Step 4: Trigger download of the zip file
files.download(zip_file_path)

File size: 247311648 bytes (235.85 MB)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Testing

Testing the model

In [13]:
# Step 8: Inference function
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    outputs = model(**inputs)
    logits = outputs.logits.detach().cpu().numpy()
    prediction = np.argmax(logits, axis=1)[0]
    return "SQL" if prediction == 1 else "Other"

In [12]:
# Example usage
test_texts = [
    "List all customers who ordered more than $100.",
    "Send an email to Alice.",
    "Users",
    "list users",
    "list me all the users with the name ashp116"
]
for text in test_texts:
    print(f"Input: {text} -> Prediction: {predict(text)}")

Input: List all customers who ordered more than $100. -> Prediction: SQL
Input: Send an email to Alice. -> Prediction: Other
Input: Users -> Prediction: SQL
Input: list users -> Prediction: SQL
Input: list me all the users with the name ashp116 -> Prediction: SQL
